Use features from the package pytket.passes to optimize the LiH and H2 example provided
UCLA Bruinium
Augest 2022

In [1]:
# import the packages
from pytket.qasm import circuit_to_qasm, circuit_from_qasm
from pytket.passes import DecomposeMultiQubitsCX, SequencePass, OptimisePhaseGadgets, CommuteThroughMultis, RemoveRedundancies, RepeatPass, RepeatWithMetricPass
from pytket.predicates import CompilationUnit
from pytket.circuit import OpType
from pytket.extensions.qiskit import AerBackend, AerStateBackend, IBMQEmulatorBackend
from pytket.architecture import Architecture
from pytket.circuit import Node

In [2]:
# import the circuit from QASM file
CircLiH = circuit_from_qasm('LiHJordanWignerMapper.qasm')
CircHH = circuit_from_qasm('H2JordanWignerMapper.qasm')

In [3]:
# Print out the number of qubits, circuit depth and depth of CX gate
def CircuitPara(circ):
    print('Qubits: '+ str(circ.n_qubits))
    print('Depth: ' + str(circ.depth()))
    print('Depth CX: ' + str(circ.depth_by_type(OpType.CX)))

In [4]:
# number of qubits and circuit depth
print('LiH Jordan Winger Mapper')
CircuitPara(CircLiH)
print('------------------------')
print('H2 Jordan Winger Mapper')
CircuitPara(CircHH)

LiH Jordan Winger Mapper
Qubits: 12
Depth: 9834
Depth CX: 7964
------------------------
H2 Jordan Winger Mapper
Qubits: 4
Depth: 83
Depth CX: 52


In [5]:
# Different cost functions
# 𝑛-qubit gates having a cost of  𝑛^2
def cost1(circ):
    return sum(pow(len(x.args), 2) for x in circ)
# cost by the depth of the circuit
def cost2(circ):
    return circ.depth()
# cost by the CX depth of the circuit
def cost3(circ):
    return circ.depth_by_type(OpType.CX)

In [6]:
# Optimize the circ with the pass1 based on the cost function
def OptiCircuit(circ,cost,pass1):
    pass2 = RepeatWithMetricPass(pass1, cost)
    cu = CompilationUnit(circ)
    pass2.apply(cu)
    return cu.circuit

In [7]:
# Compile it according to the backend
def CompToBackend(circ,backend):
    if backend.valid_circuit(circ):
        print('The optimized circuit is valid for the backend')
        return circ
    else:
        print('The optimized circuit is not valid for the backend')
        print('Now compile it')
        return backend.get_compiled_circuit(circ)

In [8]:
# Optimize the H2 example
circ = CircHH
pass1 = SequencePass([OptimisePhaseGadgets(), CommuteThroughMultis(), RemoveRedundancies()])
circ1 = OptiCircuit(circ,cost1,pass1)
print('Circuit depth after the optimization:')
CircuitPara(circ1)

Circuit depth after the optimization:
Qubits: 4
Depth: 65
Depth CX: 44


In [10]:
#b = AerBackend()
#b = AerStateBackend()
b = IBMQEmulatorBackend('ibmq_lima', hub='ibm-q', group='open', project='main') #belem, manila, quito,lima
circ2 = CompToBackend(circ1,b)
print('Final circuit parameters:')
CircuitPara(circ2)

The optimized circuit is not valid for the backend
Now compile it
Final circuit parameters:
Qubits: 4
Depth: 84
Depth CX: 47


In [11]:
# Optimize the LiH example
circ = CircLiH
pass1 = SequencePass([OptimisePhaseGadgets(), CommuteThroughMultis(), RemoveRedundancies()])
circ1 = OptiCircuit(circ,cost1,pass1)
print('Circuit depth after the optimization:')
CircuitPara(circ1)

Circuit depth after the optimization:
Qubits: 12
Depth: 8573
Depth CX: 7330


In [12]:
#b = AerBackend()
b = AerStateBackend()
#b = IBMQEmulatorBackend('ibmq_lima', hub='ibm-q', group='open', project='main') #belem, manila, quito,lima
circ2 = CompToBackend(circ1,b)
print('Final circuit parameters:')
CircuitPara(circ2)

The optimized circuit is valid for the backend
Final circuit parameters:
Qubits: 12
Depth: 8573
Depth CX: 7330
